<a href="https://colab.research.google.com/github/momo54/minfo2022/blob/main/MasterInfo_UE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Info 2022 Analysis and document generation

The overall workflow is:
* get pdf of Master on the web site
 * 12 PDF (M1/M2 + parcours)
* transform it into excel with ILovePDF
 * https://www.ilovepdf.com/fr/pdf_en_excel

* These two first steps produce a set of excel files. This is the input of notebook.
* Extract All UES from excel file with pandas
 * In this notebook
* Produce ues-uniq.csv for alignement with the ACM classif
 * At this step users can edit the ues-uniq to perform alignement from UE to ACM types
* Extract ECTS and hours from excel file
* produce the radar view thanks to tehe ACM classif


Notes:
Bon... L'extraction excel de pdf ne peut être miraculeuse. Quand un tableau est à cheval sur plusieurs pages, le tableau est mal reconstitué... Du coup, extraire les UEs des Excel est très laborieux...

# install spark for Spark SQL

In [2]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 25 kB/s 
     |████████████████████████████████| 199 kB 51.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a1fac88a59848509689648fc0533ab932ccee3e4475a68e145fca55b24b1953a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import pandas as pd

parcours=[
    ('ALMA','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'),
    ('ALMA','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx'),
    ('ATAL','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx'),
    ('ATAL','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx'),
    ('ORO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('ORO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('DS','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Data_Science_(DS)_2022-2023.xlsx'),
    ('DS','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Data_Science_(DS)_2022-2023-withcode.xlsx'),
    ('VICO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Visual_Computing_(VICO)_2022-2023.xlsx'),
    ('VICO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Visual_Computing_(VICO)-withcode.xlsx'),
    ('CMI','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_CMI-OPTIM_2022-2023.xlsx'),
    ('CMI','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_CMI-OPTIM_2022-2023.xlsx')
]

# Extracting Table UES_T from excel Master Info files

UE with description, parcours, année

In [4]:
import pandas as pd

##
## Autre Approche
## Même si 1 sheet != 1 UE
## l'odre est préservé
## regexp extraction en suivant l'ordre...

my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

df=pd.read_excel(my_csv, index_col=None, header=None, sheet_name="Table 5")
#Title and code of UE not at the right Place :(
# need swapping...
#print(df)
#print(df.dtypes)
mask=df.apply(lambda x: x.str.contains('X1.*|Objectifs|Contenu',regex=True)).any(axis=1)
print(df[mask])



                                        0  \
1                                 X1II010   
14  Objectifs (résultats d'apprentissage)   
15                                Contenu   
19                                X1II030   
32  Objectifs (résultats d'apprentissage)   
33                                Contenu   

                                                    1  
1                                             Graphes  
14  1. reconnaître et modéliser un problème réel a...  
15  •\nEléments de théorie des graphes\n•\nArbres ...  
19                          Complexité et algorithmes  
32  • Savoir manipuler les notations standards des...  
33                                                NaN  


In [5]:
#
# DS M2 et Vico M2 n'ont pas les code UE pour les UE...
# added it by hand in excel files.... -> withcode.xlsx
#


def extract_ue(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  for sheet_name, df in pd.read_excel(path, index_col=None, header=None, sheet_name=None).items():
      sheetid=int(sheet_name.split(" ")[1])
      if sheetid>4:
        print(sheet_name)
        dfs.append(df)
  allUE=pd.concat(dfs).reset_index(drop=True)

  mask=allUE.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*|Object|Contenu',regex=True)).any(axis=1)
  UES=allUE[mask]

  #debug
  #print(UES.columns.values)
  #print(UES)
  #i don't know why i need 3 colmns sometimes...
  # read_excel pb ???
  if len(UES.columns.values)==2:
    UES.set_axis(['X','Y'],axis=1,inplace=True)
  else:
    UES.set_axis(['X','Y','Z'],axis=1,inplace=True)
    
  #debug
  #print(UES.columns.values)
  #print(UES)
  #for index, row in UES.iterrows():
  #  print(f"extracting:{row['X']}")

  state=0
  key=""
  title=""
  objective=""
  content=""
  lines=[]

  import re
  reg=re.compile("^X[1-4].*|^ZZ|^YG.*")

  for index, row in UES.iterrows():
    #print(f"extracting:{row['X']}")
    #print(row['X'],row['Y']
    if state==0:
      if reg.match(str(row['X'])) is not None:
        key=row['X']
        title=row['Y']
        state=1
      else:
        print(f"error: should read an UE key:{row['X']}")
        state=0
    elif state==1:
      if str(row['X']).startswith("Object"):
        objective=row['Y']
        state=2
      else:
        print(f"error: should read an Objective:{row['X']}")
        state=0
    elif state==2:
      if str(row['X']).startswith("Contenu"):
        content=row['Y']
        lines.append({'key' : key, 'title':title,'objective':objective,'content':content,'parc':parcours,'an':an,'from':path})
        state=0
      else:
        print(f"error: should read an Content:{row['X']}")
        state=0
        
  out=pd.DataFrame(lines)
  out.to_csv(f"{parcours}_{an}_UEs.csv",sep='|',index=False,header=True)
  return out

dfs=[]
for p,a,path in parcours:
  df=extract_ue(p,a,path)
  dfs.append(df)

all=pd.concat(dfs)
print(all)


extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(A

In [6]:

# NOT WORKING -> CVS are not correctly red...
# Walk around -> not reading CVS -> Read data in memory with serializing/deserializing in CSV...

# OK CSV shoud be generated now
# Read UE in SPARK to groupby UE CODE...
# Troubles when reading CSV in spark... Some end of fields not well detected...
# Walk around on next block of code... (NO CSV)

# from pyspark.sql.types import *
# schema = StructType([
#     StructField("key",StringType(),True), 
#     StructField("title", StringType(), True),
#     StructField("objective", StringType(), True),
#     StructField("content", StringType(), True),
#     StructField("parc", StringType(), True),
#     StructField("an", StringType(), True),
#     StructField("path", StringType(), True),
#     ])

# file_list=[]
# for pa,an,path in parcours:
#   file_list.append(f"/content/{pa}_{an}_UEs.csv")

# #print(','.join(file_list))

# data = spark.read \
#     .format("csv")   \
#     .option("delimiter","|") \
#     .option("multiLine", True)\
#     .option("header",True) \
#     .schema(schema) \
#     .load(file_list)
# print(data.count())
# data.show(209)

# data.createOrReplaceTempView(f"UES")

# UES_table=spark.sql("select key,title,objective, collect_set(parc), collect_set(an) from UES  group by key,title,objective,content")

# UES_table.createOrReplaceTempView(f"UES_T")
# #don't understand why pb in CSV...
# UES_table.show(5)





# Create the table of all UEs of the master Info 

This file can be edited in Google Sheet to align with ACM classif...

In [7]:
## Work on UE data with SPARK now...

mydata=spark.createDataFrame(all.astype(str))
#mydata.show(5)

mydata.createOrReplaceTempView(f"UES")

ues=spark.sql("select key,title,objective, content, concat_ws(',',collect_set(parc)) as parcours, concat_ws(',',collect_set(an)) as an from UES  group by key,title,objective,content")
ues.show(5) 

ues.createOrReplaceTempView(f"UES_T")




+-------+--------------------+--------------------+--------------------+--------------------+---+
|    key|               title|           objective|             content|            parcours| an|
+-------+--------------------+--------------------+--------------------+--------------------+---+
|X1I2020|Graphes II et Rés...|1. reconnaître, m...|•\nProblème du fl...|ATAL,ORO,DS,CMI,VICO| M1|
|X1II040|Anglais scientifique|Au terme de cette...|Anglais de spécia...|ATAL,ORO,DS,CMI,A...| M1|
|X2II020|         Data Mining|A l'issue de ce c...|Le Data Mining es...|ATAL,ORO,DS,CMI,A...| M1|
|X1I1020|       Test logiciel|• To be able to c...|• Functional and ...|                ALMA| M1|
|X1LI010|Management à Visé...|A l’issue de cet ...|Autour d’une form...|   ATAL,DS,ALMA,VICO| M1|
+-------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 5 rows



# Write this table as a CSV file 

This file is the basis to align UEs from the Master to ACM classification.

The CSV file is not the same as the Gssheat/Excel file -> no risk to overwrite...

In [8]:
!rm -rf ues-uniq.csv

# Pandas write CSV correctly !!
ues.toPandas().to_csv('ues-uniq.csv')
#ues.write.options(header='True',delimiter='|') \
#  .csv("ues-uniq.csv")

!cp ues-uniq.csv /content/drive/MyDrive/MasterInfo2022

# Extracting ECTS_T from Excel files

For each UE in every training Course, extract:
UE -> ECTS, Hours

In [9]:
# ok now lets extracts ECTS and hours linked to UE from excel files

def extract_ects(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  data=pd.read_excel(path, index_col=None, header=1, sheet_name="Table 2")
  mask=data.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*',regex=True)).any(axis=1)
  ects=data[mask]
  #print(ects)
  return ects

dfs=[]
for p,a,path in parcours:
  df=extract_ects(p,a,path)
  dfs.append(df)

ects=pd.concat(dfs)
print(ects)



extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx
extracting ues for ORO, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for ORO, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for DS, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converte

# create the table of ECTS_T

For each UE -> how many ECTS and hours...

In [10]:
## Work with ECTS with SPARK

#from pyspark.sql.types import *
#ects_schema = StructType([
#    StructField("0", StringType(), True)\
#    ,StructField("1", StringType(), True)
#    ,StructField("2", LongType(), True)
#    ,StructField("3", StringType(), True)
#    ,StructField("4", StringType(), True)
#    ,StructField("5", StringType(), True)
#    ,StructField("6", StringType(), True)
#    ,StructField("8", LongType(), True)
#    ,StructField("9", StringType(), True)
#    ,StructField("10", StringType(), True)
#    ])

s_ects=spark.createDataFrame(ects.astype(str))
#s_ects=spark.createDataFrame(ects,schema=ects_schema)

print(f"Reading {s_ects.count()} in ECTS tablles")

s_ects=s_ects.withColumnRenamed("0","title") \
  .withColumnRenamed("Code","key") \
  .withColumnRenamed("ECTS","ects") \
  .withColumnRenamed("Total","hours") \



s_ects.createOrReplaceTempView(f"ECTS")

ects_table=spark.sql("select key,  cast(max(ects) as float) as ects, cast(max(hours) as float) as hours from ECTS  group by key,ects,hours")
ects_table.createOrReplaceTempView(f"ECTS_T")
ects_table.show(5)

print(f"produced {ects_table.count()} in ECTS_T")


ects_table.toPandas().to_csv('ects.csv')


Reading 240 in ECTS tablles
+-------+----+-----+
|    key|ects|hours|
+-------+----+-----+
|X1CI010| 2.0| 22.0|
|X1I1010| 6.0| 60.0|
|X1I1020| 3.0| 24.0|
|X1I1030| 4.0| 32.0|
|X1I1030| 5.0| 32.0|
+-------+----+-----+
only showing top 5 rows

produced 104 in ECTS_T


# Extracting ACM_T types alignement from ues_uniq.xlsx

https://docs.google.com/spreadsheets/d1OV_6-OdNZNTQ0tVyawNLPrJgMV-4ja11v8WxeqqNJfU/edit#gid=2111605593

contains all UEs with alignements to ACM Classif (CS2023)
Extract it and put in table UES_T

In [11]:
# I suppose now that UE has be aligned with ACM classif...
# Read the excel file and group yby ACM items

path='/content/drive/MyDrive/MasterInfo2022/ues-uniq.xlsx'
data=pd.read_excel(path, index_col=None, header=2)

#swtich ain spark now
s_acm=spark.createDataFrame(data.astype(str))
s_acm.show(5)

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum

#s_acm=s_acm.select("key",concat_ws("|",*[when(col(c)=="x",c).alias(c) for c in s_acm.columns]).alias("types"))
# hmmm, bricolage...
s_acm=s_acm.select("key", "title", array_remove(array([when(col(c)=="x",c).otherwise('#').alias(c) for c in s_acm.columns]),'#').alias("types"))

s_acm.show(5,truncate=200)

s_acm.createOrReplaceTempView("ACM_ARRAY")
s_acm=spark.sql("select key, title ,types, size(types) as nbtypes from ACM_ARRAY")

s_acm.show(94,truncate=200)

s_acm=s_acm.select("key",explode("types").alias("types"),"nbtypes")
s_acm.show(100,truncate=200)

s_acm.createOrReplaceTempView("ACM_T")


+----------+-------+--------------------+--------------------+--------------------+--------+---+--------------------------+------------------------------+----------+-----------------------------+---------------------------------+-------------------------------------------------+-----------+----------------------+----------------------+------------------+--------+---------------------------------+-------------+-------------------+-----------------+------------------------+-------------------------------+-------------------------------+------------------------------+-------------------------+----------------------------+------------------------+-------------------------+---------------------------------------+-----------------+-------+------------+
|Unnamed: 0|    key|               title|           objective|             content|parcours| an|T_MSF (Proof, graph proba)|T_AL: Algorithm and Complexity|T_Modeling|T_PL: prog and formal methods|CS_Computer architecture and orga|CS_Concurrent,

# Prepare data for Plotting

In [12]:
# need to compute all types and all prefix to complete results when a training course have not all types or prefix

tmptypes=spark.sql("select types, 0 as ects, 0 as hours from ACM_T group by types")
tmptypes.createOrReplaceTempView("TMPTYPES")
print(f"number of types:{tmptypes.count()}")

tmpprefix=spark.sql("select substring(ACM_T.types,1,2) as prefix, 0 as ects, 0 as hours from ACM_T group by substring(ACM_T.types,1,2)")
tmpprefix.createOrReplaceTempView("TMPPREFIX")
print(f"number of prefix:{tmpprefix.count()}")



number of types:26
number of prefix:5


In [13]:
# Bon plotting now ??

# carefull, we need to have all categories for each parcours !!
# create a temp table

# 1 M1 -> 450h et 60ects
# 1 M2 -> 261h et 60ects
# 1 M1 + M2 ->  711h et 120 ects
# j'ai compté qu'un étudiant faisait toutes les options -> il faut normaliser aux attendus pour que les parcours soient comparables

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum


results_pa={}
for pa,an,path in parcours:
  print(f"parcours:{pa}, année:{an}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by ACM_T.types
  order by ACM_T.types
  """
  
  results_pa[pa]={}
  results_pa[pa+an]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_pa[pa+an].show(26)
  print(results_pa[pa+an].count())
#  results_pa[pa][an].select(sum("ects"),sum("hours")).show()

  nbects=results_pa[pa+an].select(sum('ects')).collect()[0][0]
  nbhours=results_pa[pa+an].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_pa[pa+an]=results_pa[pa+an].withColumn("ects",col("ects")*60/nbects)
  if an=='M1':
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*450/nbhours)
  else:
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*261/nbhours)


  results_pa[pa+an].select(sum("ects"),sum("hours")).show()



parc=set([p[0] for p in parcours])

results_p={}
for pa in parc:
  print(f"parcours:{pa}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
  group by ACM_T.types
  order by ACM_T.types
  """
 
  results_p[pa]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_p[pa].show(26)
  print(results_p[pa].count())

  nbects=results_p[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_p[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_p[pa]=results_p[pa].withColumn("ects",col("ects")*120/nbects)
  results_p[pa]=results_p[pa].withColumn("hours",col("hours")*711/nbhours)

  results_p[pa].select(sum("ects"),sum("hours")).show()




parcours:ALMA, année:M1
+--------------------+------------------+------------------+
|               types|              ects|             hours|
+--------------------+------------------+------------------+
|ACS_Commputer gra...|               1.0|               8.0|
|ACS_Computational...|               0.0|               0.0|
|ACS_Image and sou...|               0.0|               0.0|
|ACS_Natural Langu...|               1.0|               8.0|
|ACS_Social comput...|               2.0|14.857142857142858|
|ACS_Software engi...|               5.5| 48.85714285714286|
|ACS_Specialized P...| 4.333333333333333|30.857142857142854|
|       AI_Classifier|0.6666666666666666| 7.996666590372722|
|            AI_Logic|               4.5|              32.0|
| AI_Machine learning|1.6666666666666665|19.996666590372723|
|   AI_Neural network|0.6666666666666666| 7.996666590372722|
|AI_Proba for unce...|               1.0|               8.0|
|  AI_Search and opti|               1.0|               8.0|


In [14]:
results_prefix={}
for pa in parc:
  print(f"prefix parcours:{pa}")

  query=f"""
    select substring(ACM_T.types,1,2) as prefix, 
        sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
        sum(ECTS_T.hours/ACM_T.nbtypes) as hours
    from ACM_T,ECTS_T, UES_T 
    where 
      ACM_T.key=ECTS_T.key 
      and ACM_T.key=UES_T.key 
      and UES_T.parcours like '%{pa}%'
    group by substring(ACM_T.types,1,2) 
    """
  results_prefix[pa]=spark.sql(query).union(tmpprefix).dropDuplicates(["prefix"]).orderBy(["prefix"])
  results_prefix[pa].show(26)
  print(results_prefix[pa].count())

  nbects=results_prefix[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_prefix[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_prefix[pa]=results_prefix[pa].withColumn("ects",col("ects")*120/nbects)
  results_prefix[pa]=results_prefix[pa].withColumn("hours",col("hours")*711/nbhours)

  results_prefix[pa].select(sum("ects"),sum("hours")).show()



prefix parcours:CMI
+------+------------------+------------------+
|prefix|              ects|             hours|
+------+------------------+------------------+
|    AC|              15.5|             128.0|
|    AI|              18.0|178.65666643778485|
|    CS|               8.0|              68.0|
|    SO|              70.0|             196.0|
|    T_|61.499999999999986| 581.3333333333334|
+------+------------------+------------------+

5
ects:173.0
hours:1151.9899997711182
+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|119.99999999999999|     711.0|
+------------------+----------+

prefix parcours:DS
+------+------------------+------------------+
|prefix|              ects|             hours|
+------+------------------+------------------+
|    AC|18.066666666666663| 138.9714285714286|
|    AI|              25.4|209.55666643778486|
|    CS|24.033333333333335| 185.7952380952381|
|    SO|33.333333333333336|217.50000000000003|
|    T_|2

# Plotting

In [15]:
# Radar plot Now

!pip install -U kaleido

import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

folder="/content/drive/MyDrive/MasterInfo2022/plots/"

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories=flatten(results_p['ALMA'].select("types").collect())

pdata=[]
for p in parc:
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

fig_h.write_image(f"{folder}MasterInfoHours.pdf",engine="kaleido")

pdata=[]
for p in parc:
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")

fig_e.write_image(f"{folder}MasterInfoHours.pdf",engine="kaleido")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 153 kB/s 


# plotting prefix

In [16]:
import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories_p=flatten(results_prefix['ALMA'].select("prefix").collect())

pdata=[]
for p in parc:
  h=results_prefix[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories_p, name=p,mode="lines+markers+text")
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

fig_h.write_image(f"{folder}MasterBlockInfoHours.pdf",engine="kaleido")


pdata=[]
for p in parc:
  h=results_prefix[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories_p, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")

fig_e.write_image(f"{folder}MasterBlockInfoECTS.pdf",engine="kaleido")


# plotting per Parcours


In [17]:

# https://plotly.com/python/polar-chart/
for p in parc:

  results_p[p].select(sum("ects"),sum("hours")).show()

  pdata=[]
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()

  print(f"hours:{h}")

  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Master Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 160]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  fig.write_image(f"{folder}Master-{p}-Hours.pdf",engine="kaleido")


  pdata=[]
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  print(f"ects:{h}")
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Master Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 35]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  fig.write_image(f"{folder}Master-{p}-ECTS.pdf",engine="kaleido")


+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|120.00000000000003|     711.0|
+------------------+----------+

hours:[4.937542861596119, 59.25051433915343, 0.0, 4.937542861596119, 4.937542861596119, 0.0, 4.937542861596119, 24.685656951366855, 0.0, 12.341799797376556, 4.935485504982377, 4.937542861596119, 63.365133390483535, 9.875085723192239, 9.875085723192239, 7.406314292394179, 14.812628584788358, 0.0, 26.53929288107914, 70.97717863544422, 16.04701430018739, 7.406314292394179, 157.1784477608098, 92.99039056006026, 79.00068578553791, 29.625257169576717]


ects:[0.6936416184971099, 7.976878612716765, 0.0, 0.6936416184971099, 0.6936416184971099, 0.0, 0.6936416184971099, 3.5838150289017348, 0.0, 1.1560693641618498, 0.46242774566474, 0.6936416184971099, 6.589595375722545, 1.3872832369942198, 1.3872832369942198, 0.6936416184971099, 2.0809248554913298, 0.0, 9.47976878612717, 19.53757225433526, 15.375722543352602, 4.1618497109826595, 18.150289017341045, 10.867052023121389, 10.520231213872835, 3.1213872832369947]


+------------------+-----------------+
|         sum(ects)|       sum(hours)|
+------------------+-----------------+
|119.99999999999999|710.9999999999999|
+------------------+-----------------+

hours:[11.38750137899918, 17.08125206849877, 2.277500275799836, 11.38750137899918, 27.655360491855156, 4.880357733856791, 24.239110078155395, 42.1313826518765, 22.77500275799836, 44.9782579966263, 11.385128928578718, 17.08125206849877, 10.794402348842972, 16.267859112855973, 21.961609802355564, 72.04289190874572, 17.08125206849877, 4.880357733856791, 71.05326381271364, 39.85625482649713, 14.115756917717734, 29.773571313841607, 56.9375068949959, 59.784382239745696, 25.02877907259195, 34.16250413699754]


ects:[1.8604651162790697, 3.1007751937984493, 0.18604651162790697, 1.8604651162790697, 4.651162790697675, 0.9302325581395349, 4.217054263565892, 6.852713178294573, 4.186046511627907, 6.387596899224806, 1.5503875968992247, 2.7906976744186047, 1.8604651162790697, 2.7906976744186047, 3.7209302325581395, 12.124031007751936, 2.7906976744186047, 0.9302325581395349, 11.472868217054264, 7.131782945736434, 6.821705426356589, 5.5813953488372094, 7.906976744186046, 10.232558139534884, 3.875968992248062, 4.186046511627907]


+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|120.00000000000001|     711.0|
+------------------+----------+

hours:[28.966934885165607, 10.533430867332948, 46.347095816264975, 5.266715433666474, 33.48126239973687, 4.514327514571264, 30.321233139536986, 41.07818586760716, 21.066861734665896, 27.911397283440973, 17.904637959474673, 5.266715433666474, 5.266715433666474, 15.047758381904213, 15.047758381904213, 46.121379440536415, 15.800146300999423, 9.781042948237738, 56.6171909119146, 36.86700803566532, 61.35723480221442, 18.43350401783266, 52.667154336664744, 55.30051205349798, 18.43350401783266, 31.600292601998845]


...

ects:[0.7643312101910829, 8.789808917197453, 0.0, 0.7643312101910829, 0.7643312101910829, 0.0, 0.7643312101910829, 3.949044585987261, 0.0, 1.273885350318471, 0.5095541401273885, 0.7643312101910829, 7.261146496815287, 1.5286624203821657, 1.5286624203821657, 0.7643312101910829, 2.2929936305732483, 0.0, 8.152866242038218, 17.70700636942675, 16.942675159235666, 1.5286624203821657, 20.0, 8.9171974522293, 11.59235668789809, 3.4394904458598727]


+------------------+-----------------+
|         sum(ects)|       sum(hours)|
+------------------+-----------------+
|119.99999999999997|711.0000000000001|
+------------------+-----------------+

hours:[6.070502354763048, 0.0, 0.0, 6.070502354763048, 11.273790087417089, 77.29050319546523, 32.52054832908776, 6.0679729208890745, 24.28200941905219, 15.173726453033645, 6.0679729208890745, 6.070502354763048, 6.070502354763048, 43.14392744992309, 58.32018333683071, 59.837808925521465, 22.764383830361428, 5.203287732654041, 52.35808280983128, 60.705023547630475, 10.623379120835333, 26.55844780208833, 39.458265305959806, 27.317260596433712, 68.29315149108429, 39.458265305959806]


ects:[0.7894736842105262, 0.0, 0.0, 0.7894736842105262, 1.5789473684210524, 9.671052631578945, 4.605263157894736, 0.5263157894736841, 3.5526315789473677, 1.3157894736842102, 0.5263157894736841, 0.7894736842105262, 0.7894736842105262, 5.723684210526315, 7.69736842105263, 8.157894736842103, 2.960526315789473, 0.7894736842105262, 10.789473684210526, 18.684210526315788, 17.10526315789473, 4.210526315789473, 4.078947368421051, 3.026315789473683, 7.763157894736839, 4.078947368421051]


+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|120.00000000000001|     711.0|
+------------------+----------+

hours:[21.444122946406818, 10.722061473203409, 31.09397827228989, 35.15942658087951, 26.039292149208283, 7.811787644762484, 30.864219812149813, 45.030424373520624, 21.444122946406818, 35.64862058446765, 25.462662184924408, 5.3610307366017045, 9.381803789052983, 15.317230676004872, 15.317230676004872, 54.98885812685749, 16.083092209805113, 4.595169202801461, 25.464895998858097, 50.03628687494924, 42.79889538053694, 18.763607578105965, 61.651853470919605, 45.56876126111449, 22.784380630557244, 32.166184419610225]


ects:[2.3529411764705888, 1.3725490196078431, 3.294117647058824, 3.3921568627450984, 2.9411764705882355, 0.9411764705882354, 3.549019607843138, 4.921568627450981, 2.647058823529412, 3.3039215686274512, 2.598039215686275, 0.5882352941176472, 1.0294117647058825, 1.7647058823529413, 1.7647058823529413, 6.196078431372549, 1.7647058823529413, 0.5882352941176472, 12.54901960784314, 21.176470588235297, 16.607843137254903, 8.823529411764707, 5.588235294117648, 5.294117647058824, 2.3039215686274512, 2.647058823529412]


In [18]:
# https://plotly.com/python/polar-chart/
for p in parc:

  pdata=[]
  for an in ['M1','M2']:
    h=results_pa[p+an].select("hours").rdd.map(lambda x: x.hours).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Master Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 100]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  fig.write_image(f"{folder}Master-{p}-py-hours.pdf",engine="kaleido")


  pdata=[]
  for an in ['M1','M2']:
    h=results_pa[p+an].select("ects").rdd.map(lambda x: x.ects).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Master Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 20]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  fig.write_image(f"{folder}Master-{p}-py-ects.pdf",engine="kaleido")


In [19]:
#Debug, Debug...

#spark.sql("select * from UES_T where UES_T.parcours like '%DS%' and UES_T.an = 'M2'").show()
#spark.sql("select * from ACM_T where key like '%ZZ%'").show() 
#spark.sql("select * from ECTS_T where key like '%ZZ%'").show() 

## CMD + Crt + '+' to comment/uncomment
## Control ATAL
## in fact OK
# pa='ATAL'
# an='M2'
# query=f"""
#   select ACM_T.types, 
#       sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
#       sum(ECTS_T.hours/ACM_T.nbtypes) as hours
#   from ACM_T,ECTS_T, UES_T 
#   where 
#     ACM_T.key=ECTS_T.key 
#     and ACM_T.key=UES_T.key 
#     and UES_T.parcours like '%{pa}%'
#     and UES_T.an = '{an}'
#   group by ACM_T.types
#   order by ACM_T.types
#   """
  
# spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"]).show()

pa='ATAL'
an='M2'
query=f"""
  select substring(ACM_T.types,1,2) as prefix, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by substring(ACM_T.types,1,2) 
  """
  
spark.sql(query).show()

+------+------------------+------------------+
|prefix|              ects|             hours|
+------+------------------+------------------+
|    SO| 71.23333333333333| 68.53333333333333|
|    T_|              1.75|              18.0|
|    AI|11.249999999999998| 89.99999999999999|
|    CS|               1.5|              12.0|
|    AC|18.266666666666666|152.46666666666664|
+------+------------------+------------------+



# Generate the competencies file (types-ues.csv)

In [20]:

# types to UES...
# help fill competencies

spark.sql("select * from ACM_T").show(5,truncate=200)

# bon elles sont bien là toutes les croix non ??
spark.sql("select * from ACM_T where ACM_T.types='ACS_Computational science'").show(20,truncate=200)

# bon elles sont bien là toutes les croix non ??
spark.sql("select ACM_T.types, concat_ws('\n',collect_set(ACM_T.key))  from ACM_T where ACM_T.types='ACS_Computational science' group by ACM_T.types").show(20,truncate=200)


#spark.sql("select ACM_T.types, concat_ws("\n",collect_set(ACM_T.key) as UEs from ACM_T group by ACM_T.types").show(5,truncate=200)


query=f"""
  select ACM_T.types, 
       "to write !!" as Competencies,
       concat_ws("\n",collect_set(concat_ws(' ',UES_T.title,UES_T.key))) as UEs,
       concat_ws("\n",collect_set(concat_ws("\n",UES_T.title,UES_T.key,UES_T.objective,UES_T.content))) as Objectives
  from ACM_T, UES_T 
  where 
    ACM_T.key=UES_T.key 
  group by ACM_T.types 
  order by ACM_T.types
  """
  
types_to_ues=spark.sql(query)
types_to_ues.toPandas().to_csv('types-ues-gen.csv')

#write also to my drive...
types_to_ues.toPandas().to_csv('/content/drive/MyDrive/MasterInfo2022/types-ues-gen.csv')

types_to_ues.show(5,truncate=200)

+-------+------------------------------+-------+
|    key|                         types|nbtypes|
+-------+------------------------------+-------+
|X1I1010|                    T_Modeling|      2|
|X1I1010|      ACS_Software engineering|      2|
|X1I1020| T_PL: prog and formal methods|      2|
|X1I1020|      ACS_Software engineering|      2|
|X1I1060|T_AL: Algorithm and Complexity|      2|
+-------+------------------------------+-------+
only showing top 5 rows

+-------+-------------------------+-------+
|    key|                    types|nbtypes|
+-------+-------------------------+-------+
|X1I2020|ACS_Computational science|      3|
|  ZZSKR|ACS_Computational science|      3|
|X1I2010|ACS_Computational science|      3|
|X1II070|ACS_Computational science|      3|
|X3IO040|ACS_Computational science|      3|
|X3IO050|ACS_Computational science|      3|
|X3IO060|ACS_Computational science|      3|
|X3IO080|ACS_Computational science|      2|
|X3IOM20|ACS_Computational science|      3|
|X3IOM